In [31]:
import pandas as pd

In [32]:
data = pd.read_csv("cleanresult.csv", header=0, delimiter=",")

data['lsentiment']=0

In [33]:
print data.shape

(112197, 30)


Taking sentiment of data using Sentiwordnet method

In [34]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [35]:
lemmatizer = WordNetLemmatizer()

In [36]:
def penn_to_wn(tag):
     if tag.startswith('J'):
        return wn.ADJ
     elif tag.startswith('N'):
        return wn.NOUN
     elif tag.startswith('R'):
        return wn.ADV
     elif tag.startswith('V'):
        return wn.VERB
     return None

In [37]:
def swn_polarity(text):
    sentiment = 0
    tokens_count = 0
  
    for raw_sentence in text:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
            
            if not tokens_count:
                return 0
            if sentiment >= 0:
                return 1
            
            return 0

In [38]:
textdata1 = data["reviewText"]

In [39]:
lsentiment=[]

for i in textdata1:
    lsentiment.append(swn_polarity(i))

In [40]:
resultsentiment=zip(textdata1,lsentiment)

In [41]:
finalres=zip(resultsentiment)


In [42]:
data['lsentiment']=pd.Series(lsentiment)
print "done"

done


In [43]:
#Prediction : Creating model

#Run a logistic regression
import statsmodels.api as sm2

In [44]:
#creating training set and test set
train = data[:90000] 
test = data[90000:] 

In [63]:
#make prediction model
col= ['lsentiment']
train[col]
logit=sm2.Logit(train['confidence'],train[col])
result=logit.fit() 
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.071689
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:             confidence   No. Observations:                90000
Model:                          Logit   Df Residuals:                    89999
Method:                           MLE   Df Model:                            0
Date:                Mon, 27 Feb 2017   Pseudo R-squ.:                  -4.217
Time:                        01:25:50   Log-Likelihood:                -6452.0
converged:                       True   LL-Null:                       -1236.7
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
lsentiment     4.4589      0.033    136.079      0.000         4.395     4.523


In [68]:
pre = result.predict(test[col])
pre = (pre > 0.50).astype(int)
print(pre)

[1 1 1 ..., 1 0 1]


C:\Users\Mona\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app


Confusion Matrix


ValueError: Can't handle mix of binary and continuous